In [1]:
# test_linear_complexity.py

from o1_Acc_Feature_Extract import FeatureExtract

def main():
    # Sample binary data strings
    binary_data_list = [
        "1100101010110101" * 31,  # 500 bits
        "1111000011110000" * 31,  # 500 bits
        "1010101010101010" * 31,  # 500 bits
        "",  # Empty string to test error handling
        "0000000000000000" * 31   # 500 bits
    ]

    # Compute linear complexity features
    results = FeatureExtract.linear_complexity_test_batch(
        serial_binary_data_list=binary_data_list,
        block_size=128
    )

    # Print the results
    for idx, result in enumerate(results):
        print(f"Result for binary data {idx + 1}:")
        if 'error' in result:
            print(f"  Error: {result['error']}")
        else:
            print(f"  P-Value: {result['p_value']}")
            print(f"  Mean Complexity: {result['mean_complexity']}")
            print(f"  Standard Deviation: {result['std_complexity']}")
            print(f"  Chi-Squared Statistic: {result['chi_squared_statistic']}")
        print("-" * 50)

if __name__ == "__main__":
    main()

2024-12-08 03:36:51,088 - WARNING - Binary data at index 3 is invalid or empty.


Result for binary data 1:
  P-Value: 0.808826870907974
  Mean Complexity: 16.0
  Standard Deviation: 0.0
  Chi-Squared Statistic: 3.0001590251922607
--------------------------------------------------
Result for binary data 2:
  P-Value: 0.808826870907974
  Mean Complexity: 5.0
  Standard Deviation: 0.0
  Chi-Squared Statistic: 3.0001590251922607
--------------------------------------------------
Result for binary data 3:
  P-Value: 0.808826870907974
  Mean Complexity: 2.0
  Standard Deviation: 0.0
  Chi-Squared Statistic: 3.0001590251922607
--------------------------------------------------
Result for binary data 4:
  Error: Invalid or empty binary data.
--------------------------------------------------
Result for binary data 5:
  P-Value: 0.808826870907974
  Mean Complexity: 0.0
  Standard Deviation: 0.0
  Chi-Squared Statistic: 3.0001590251922607
--------------------------------------------------


In [32]:
import unittest
from o1_Acc_Feature_Extract import FeatureExtract
import torch
import numpy as np

class TestLongestOneBlock(unittest.TestCase):
    def setUp(self):
        # Initialize sample binary data for feature extraction
        self.binary_data_list = [
            "1111000011110000" * 100,  # 1,600 bits with runs of 4
            "1010101010101010" * 100,  # 1,600 bits with runs of 1
            "1111111111111111" * 100,  # 1,600 bits with runs of 16 (max run)
            "0000000000000000" * 100   # 1,600 bits with no ones
        ]
        
        # Initialize features_list with empty dictionaries
        self.features_list = [{} for _ in self.binary_data_list]

    def test_feature_extraction(self):
        # Extract features
        results = FeatureExtract.longest_one_block_test_batch(
            binary_data_list=self.binary_data_list,
            verbose=False  # Set to True to see debug statements
        )
        
        # Update features_list based on the results
        for i, result in enumerate(results):
            if result is not None:
                self.features_list[i].update({
                    f"longest_one_block_{key}": value.tolist() if isinstance(value, torch.Tensor) else value 
                    for key, value in result.items()
                })
            else:
                self.features_list[i]['longest_one_block_error_message'] = 'Failed to compute longest one block.'
        # print(self.features_list)
        # Define expected keys
        expected_keys = {
            'longest_one_block_p_value',
            'longest_one_block_is_random',
            'longest_one_block_frequencies',
            'longest_one_block_expected_frequencies',
            'longest_one_block_block_count',
            'longest_one_block_block_size',
            'longest_one_block_error_message'
        }
        
        for feature in self.features_list:
            # Check all expected keys are present
            self.assertTrue(expected_keys.issubset(feature.keys()), "Missing expected feature keys.")
            
            # Ensure no error messages are present
            self.assertIsNone(feature['longest_one_block_error_message'], "Unexpected error in feature extraction.")
        
        # Specific assertions for each scenario
        for feature in self.features_list:
            self.assertEqual(feature['longest_one_block_block_size'], 8, "Incorrect block size.")
            self.assertEqual(feature['longest_one_block_block_count'], 200, "Incorrect block count.")

    def test_empty_string(self):
        # Test with an empty string
        empty_data = [""]
        result = FeatureExtract.longest_one_block_test_batch(binary_data_list=empty_data, verbose=False)
        
        self.assertEqual(len(result), 1, "Should return one result.")
        self.assertIsNotNone(result[0], "Result should not be None.")
        self.assertEqual(result[0]['error_message'], 'Error: Not enough data to run this test.', "Incorrect error message for empty input.")
        self.assertEqual(result[0]['p_value'], 0.0, "p_value should be 0.0 for insufficient data.")
        self.assertFalse(result[0]['is_random'], "is_random should be False for insufficient data.")

    def test_invalid_characters(self):
        # Test with invalid characters
        invalid_data = ["11002" * 100]  # 500 bits with invalid '2's
        result = FeatureExtract.longest_one_block_test_batch(binary_data_list=invalid_data, verbose=False)
        
        self.assertEqual(len(result), 1, "Should return one result.")
        self.assertIsNotNone(result[0], "Result should not be None.")
        self.assertEqual(result[0]['error_message'], 'Error: Binary data contains invalid characters.', "Incorrect error message for invalid characters.")
        self.assertEqual(result[0]['p_value'], 0.0, "p_value should be 0.0 for invalid characters.")
        self.assertFalse(result[0]['is_random'], "is_random should be False for invalid characters.")

if __name__ == '__main__':
    unittest.main(argv=[''], verbosity=2, exit=False)

test_empty_string (__main__.TestLongestOneBlock.test_empty_string) ... FAIL
test_feature_extraction (__main__.TestLongestOneBlock.test_feature_extraction) ... ok
test_invalid_characters (__main__.TestLongestOneBlock.test_invalid_characters) ... FAIL

FAIL: test_empty_string (__main__.TestLongestOneBlock.test_empty_string)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\santh\AppData\Local\Temp\ipykernel_21204\252816921.py", line 66, in test_empty_string
    self.assertEqual(result[0]['error_message'], 'Error: Not enough data to run this test.', "Incorrect error message for empty input.")
AssertionError: 'Error: Not enough data to run this test' != 'Error: Not enough data to run this test.'
- Error: Not enough data to run this test
+ Error: Not enough data to run this test.
?                                        +
 : Incorrect error message for empty input.

FAIL: test_invalid_characters (__main__.TestLongestOn